We want to solve the linear system $$Ax=b$$ and we split the matrix into first unknown matrices $M$ and $N$
as follows 
$$A=M-N$$
where we assume that $M$ is invertible. We can then write our linear system as follows
\begin{align}
Ax&=b\\
\Leftrightarrow&\quad (M-N)x=b\\
\Leftrightarrow&\quad  Mx-Nx=b\\
\Leftrightarrow&\quad  Mx=b+Nx\\
\Leftrightarrow&\quad  x=M^{-1}b+M^{-1}Nx\\
\Leftrightarrow&\quad  x=M^{-1}b+M^{-1}(M-A)x\\
\Leftrightarrow&\quad  x=M^{-1}b+(I-M^{-1}A)x\\
\Leftrightarrow&\quad  x=x+M^{-1}(b-Ax)\\
\end{align}
so far it seems nothing is gained but let's turn this into an iteration
$$
x^{(k+1)}=x^{(k)}+M^{-1}(b-Ax^{(k)}).
$$
This method does work quite well under certain circumstances. We now want to create an actual example and for that choose 
$$
M=D
$$
with $D$ the diagonal of $A$. This is the so-called Jacobi iteration. Please implement a function that given a right hand side and a matrix implements this method. You can use the norm of the residual as a measure of convergence, i.e.,
$$
r^{(k)}=b-Ax^{(k)}.
$$
You can start with the zero vector for $x^{(0)}$.


In [1]:
import numpy as np
import scipy
from scipy import linalg as la
from scipy import optimize
import scipy.sparse as spsp

In [2]:
# function should return solution x and 
# number of iteration, such it can be called as
# (x,it)=jacobi(A,b,maxiter,tol,verbose)
# if verbose==0 the function should not print out messages
def jacobi(A,b,maxiter,tol,verbose):
    n = A.shape[0]
    if n != A.shape[1] or verbose == 0:
        return 0
    else:
        xold= np.zeros((n,1))
        x = xold #first iteration
        k=0    
        #split matrix into A = M - N
        N = np.diag(A)
        M = np.diagflat(N)
        N = M-A
        res = np.sum(abs(A@x - b)) #alternativ ueber die Norm
        while res > tol:
            if k == maxiter:
                break
            else:
                xold = x
                x = xold + la.inv(M)@(b-A@xold)
                k = k + 1
                res = np.sum(abs(A@xold - b))
    
    
        return (x,k)
    

Now check out if our iteration converges, at first with random matrx and random right hand side

In [3]:
n=5
x = np.zeros((n,1))
diag = np.diag([10,10,10,10,10])
A=np.random.rand(n,n)+diag
b = np.random.rand(n,1)

In [4]:
print(np.sum(abs(A@x - b)))
#np.sum((abs(A@x - b) > err).astype(int)) == n

2.0783278499110245


In [5]:
(x,it)=jacobi(A,b,1000,1e-5,verbose=1)
print(x)
print(it)
print(np.sum(abs(A@x - b)))

[[0.02469992]
 [0.01534037]
 [0.03333036]
 [0.00487227]
 [0.08675295]]
9
1.4817461863741999e-06


It seems not to converge, lets try another matrix, for example we make A diagonal dominant by replacing 
$A_{ii}$ with the sum of row $i$.



In [6]:
A2=A.copy()
n1=np.ones((n,1))
rs=A2@n1
#print(n1)
#print(rs)

for i in range (n):
    A2[i,i]=rs[i]
               
(x,it)=jacobi(A2,b,1000,1e-10,verbose=1)
print(np.sum(abs(A@x - b)))
print(it)

0.3209112633099818
15


The Jacobi iteration computes the vector 
$ x^{(k+1)} $ from values of vector $ x^{(k)} $. 
Now the idea comes up to use for the computation of the component $i$ of $x^{(k+1)}$ the already computed components $x^{(k+1)}_1 \cdots x^{(k+1)}_{i-1}$ from the new vector $x^{(k+1)}$  instead the components of the old vector $x^{(k)}$.

This leads to the Gauss-Seidel iteration,
instead of choosing
$$ M=D $$ with $D$ the diagonal of $A$ in the Jacobi iteration now choose
$$ M=(D+L) $$ where $L$ is the lower triangle of $A$.

Please implement again a function that given a right hand side and a matrix implements this method. 
You can use the norm of the residual as a measure of convergence, i.e.,
$$
r^{(k)}=b-Ax^{(k)}.
$$
You can start with the zero vector for $x^{(0)}$.


In [7]:
# function should return solution x and 
# number of iteration, such it can be called as
# (x,it)=gauss_seidel(A,b,maxiter,tol,verbose)
# if verbose==0 the function should not print out messages
def gauss_seidel(A,b,maxiter,tol,verbose):
    n = A.shape[0]
    if n != A.shape[1] or verbose == 0:
        return 0
    else:
        
    
        xold=np.zeros((n,1)) # dummy - to remove
        k=0
        N = np.diag(A)
        D = np.diagflat(N) #Diagonal Matrix
        L = np.tril(A, k=-1) #L is lower triangle
        M = D + L
    
    
        x = xold #first iteration   
        #split matrix into A = M - N
        res = np.sum(abs(A@x - b)) #alternativ ueber die Norm
        while res > tol:
            if k == maxiter:
                break
            else:
                xold = x
                x = xold + la.inv(M)@(b-A@xold)
                k = k + 1
                res = np.sum(abs(A@xold - b))
    
    
        return (x,k)


In [8]:
(x,it)=gauss_seidel(A2,b,1000,1e-5,verbose=1)
print(x)
print(it)
print(np.sum(abs(A2@x - b)))

[[0.02074505]
 [0.01365535]
 [0.02932432]
 [0.00514865]
 [0.07064779]]
5
2.2487496739720392e-07


Now we want to examine the convergence of Jacobi and Gauss-Seidel iteration for different system sizes.
Create a plot of iteration numbers for system sizes $n=5,10,15,20,25,30,40,50$. Use a matrix $A$ and vector $b$
of your choice but ensure that this matrix will converge at all.

In [15]:
# get the plot lib 
import matplotlib.pyplot as plt
#Maatrix A and vector b of my choice
n=50
x = np.zeros((n,1))
diag = np.diag(np.ones(n)*30)
A=np.random.rand(n,n)+diag
b = np.random.rand(n,1)

def jac(A,b,m):
    xold= np.zeros((m,1))
    x = xold #first iteration
    N = np.diag(A)
    M = np.diagflat(N)
    N = M-A
    for k in range(1,m):
        xold = x
        x = xold + la.inv(M)@(b-A@xold)
        res = np.sum(abs(A@xold - b))
    
    
    return res

num = [5,10,15,20,25,30,40,50]

#plt.show
    
    

In [21]:
jac(A,b,50)

0.0007220839929577283